# **Analisis Sentimen Twitter**
### Caesar

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Preparation

### 1.1. Import Library

In [4]:
import csv
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import ast
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords

# Libraries
import matplotlib.pyplot as plt

# Preliminaries
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

# Models
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Training
import torch.optim as optim

# Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, recall_score, precision_score
import seaborn as sns


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### 1.2. Get Data and Data Cleaning

In [5]:
# Ambil Data
df1 = pd.read_csv("T1.csv" , skiprows=8)
df2 = pd.read_csv("T2.csv" , skiprows=8)
# df3 = pd.read_csv("T3.csv" , skiprows=8)
# df4 = pd.read_csv("T4.csv" , skiprows=8)
# df5 = pd.read_csv("T5.csv" , skiprows=8)

# frames = [df1, df2, df3, df4, df5]

frames = [df1, df2]

df = pd.concat(frames)

df.rename(columns = {'Title':'text'}, inplace = True)
df['label_target'] = np.random.choice([0, 1], df.shape[0]) #masih random
df = df[['text', 'label_target']]

In [6]:
df.info

<bound method DataFrame.info of                                                    text  label_target
0     Gimana nih portofolio aset kripto kamu saat in...             0
1     Mau dapetin total hadiah USDT senilai Rp2 juta...             0
2     Hai!\n\nSembari iseng-iseng di weekend kali in...             0
3     @MetaMaskSupport hi min, I transferred matic f...             0
4     เอกลาภ Zipmex แจงกรณีถอนเหรียญ ยันไม่ปิดกระดาน...             1
...                                                 ...           ...
2900  🇮🇩 USDT/IDR and USD/IDR Price Comparison🗺\n  🇮...             1
2901  Indodax Market Signal 10 Oktober 2022 | Koin B...             1
2902  @econtikkie @YouTube เงินเกษียณผมก็อยู่ใน zipm...             0
2903  RT @Skylink1981 #PayAttention #GroveToken \n\n...             1
2904  RT @Skylink1981 #PayAttention #GroveToken \n\n...             1

[7234 rows x 2 columns]>

In [7]:
# Membersihkan data
df = df.loc[df['text'].str.contains('okocrypto') == True]
df = df.loc[df['text'].str.contains('BestAsk Exchange') == False]
df = df.loc[df['text'].str.contains('@.*@.*') == False]
df = df.loc[df['text'].str.contains('#.*#.*') == False]
df = df.loc[df['text'].str.contains('^RT ') == False]
df = df.loc[df['text'].str.contains('you ') == False]
df = df.loc[df['text'].str.contains('Why') == False]
df = df.loc[df['text'].str.contains('How') == False]
df = df.loc[df['text'].str.contains('Market.*$.*') == False]
df = df.loc[df['text'].str.contains('Hits.*$.*') == False]
df = df.loc[df['text'].str.contains('Price.*$.*') == False]
df = df.loc[df['text'].str.contains(r'[\u263a-\U0001f645].*[\u263a-\U0001f645]') == False]

df.info

<bound method DataFrame.info of                                                    text  label_target
6     Berdasarkan hasil peninjauan, Tokocrypto memut...             0
40    2. Tokocrypto\nPlatform perdagangan aset kript...             0
52    wallet kepercayaan udah bisa busd bep20. tokoc...             1
68    Tokocrypto hari ini umumkan akan hapus ETF Tok...             1
74    @scarletwitchwin Hai Kak, Kami informasikan ba...             1
...                                                 ...           ...
2875  @Tokocrypto Selamat pagi, hari ini $TKO masih ...             0
2880               Merasa kasihan dengan apk tokocrypto             1
2888                           @dogespromote Tokocrypto             1
2891                           @dogespromote Tokocrypto             0
2892  @Tokocrypto kabar pengembang token tko tewas m...             1

[555 rows x 2 columns]>

In [8]:
# Cek duplikasi
print("tweet duplicated all column",df.duplicated().sum())
print("tweet duplicated by text",df.duplicated(subset=['text']).sum())
print("total",df.duplicated().sum() + df.duplicated(subset=['text']).sum())

tweet duplicated all column 4
tweet duplicated by text 9
total 13


In [9]:
#Menghilangkan duplikasi tweet
print("before",df.shape)
df = df.drop_duplicates()
df = df.drop_duplicates('text', keep='last')
print("after",df.shape)

before (555, 2)
after (546, 2)


In [10]:
#Menghapus Duplikat berdasarkan 7 Kata awal yang sama. Code ini untuk menghapus spam yang tidak terdeteksi.

idx = []
for i in range(df.shape[0]):
    #print(i)
    if i == (df.shape[0] - 1):
        break
    temp_a = df.iloc[i].text.split()[:7]
    temp_b = df.iloc[i+1].text.split()[:7]
    if temp_a == temp_b:
        idx.append(i+1)

df.info

<bound method DataFrame.info of                                                    text  label_target
6     Berdasarkan hasil peninjauan, Tokocrypto memut...             0
40    2. Tokocrypto\nPlatform perdagangan aset kript...             0
52    wallet kepercayaan udah bisa busd bep20. tokoc...             1
68    Tokocrypto hari ini umumkan akan hapus ETF Tok...             1
74    @scarletwitchwin Hai Kak, Kami informasikan ba...             1
...                                                 ...           ...
2859  @Tokocrypto Hasil anak bangsa...koin scam http...             1
2875  @Tokocrypto Selamat pagi, hari ini $TKO masih ...             0
2880               Merasa kasihan dengan apk tokocrypto             1
2891                           @dogespromote Tokocrypto             0
2892  @Tokocrypto kabar pengembang token tko tewas m...             1

[546 rows x 2 columns]>

In [11]:
df.text.to_list()

['Berdasarkan hasil peninjauan, Tokocrypto memutuskan untuk menghapus & menghentikan perdagangan pada semua pasangan perdagangan untuk token berikut mulai 3 November 2022:\nTORN (Tornado Cash)\nXMR (Monero)\nZEC (Zcash)\n\n👉 https://t.co/KBrk6BlWVF\n\nSalam to the Moon! https://t.co/WSeokIjQwF',
 '2. Tokocrypto\nPlatform perdagangan aset kripto itu mengumumkan mengurangi 20 persen dari total 225 karyawannya atau sekitar 45 orang pada 21 September 2022. https://t.co/ENHz8TON1e',
 'wallet kepercayaan udah bisa busd bep20. tokocrypto belom nih.',
 'Tokocrypto hari ini umumkan akan hapus ETF Token LINK UP, LINK DOWN, TRX UP, dan TRX DOWN.\n\nTapi nanti pada hari Senin 10 Oktober 2022 pukul 00.00 - 24.00 WIB ETF Token LINK UP, LINK DOWN, TRX UP, dan TRX DOWN bakal dibuka sementara pada pukul 00.00 - 24.00 WIB agar bisa dijual.',
 '@scarletwitchwin Hai Kak, Kami informasikan bahwa Tokocrypto mewajibkan bagi setiap nasabah untuk mengaktifkan 2FA GA dan SMS per 15 Desember 2021 baik untuk pena

In [12]:
df['label_target'].value_counts()

0    273
1    273
Name: label_target, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 6 to 2892
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          546 non-null    object
 1   label_target  546 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.8+ KB


In [14]:
df.text.to_list()

['Berdasarkan hasil peninjauan, Tokocrypto memutuskan untuk menghapus & menghentikan perdagangan pada semua pasangan perdagangan untuk token berikut mulai 3 November 2022:\nTORN (Tornado Cash)\nXMR (Monero)\nZEC (Zcash)\n\n👉 https://t.co/KBrk6BlWVF\n\nSalam to the Moon! https://t.co/WSeokIjQwF',
 '2. Tokocrypto\nPlatform perdagangan aset kripto itu mengumumkan mengurangi 20 persen dari total 225 karyawannya atau sekitar 45 orang pada 21 September 2022. https://t.co/ENHz8TON1e',
 'wallet kepercayaan udah bisa busd bep20. tokocrypto belom nih.',
 'Tokocrypto hari ini umumkan akan hapus ETF Token LINK UP, LINK DOWN, TRX UP, dan TRX DOWN.\n\nTapi nanti pada hari Senin 10 Oktober 2022 pukul 00.00 - 24.00 WIB ETF Token LINK UP, LINK DOWN, TRX UP, dan TRX DOWN bakal dibuka sementara pada pukul 00.00 - 24.00 WIB agar bisa dijual.',
 '@scarletwitchwin Hai Kak, Kami informasikan bahwa Tokocrypto mewajibkan bagi setiap nasabah untuk mengaktifkan 2FA GA dan SMS per 15 Desember 2021 baik untuk pena

In [15]:
df.to_csv("dataset_skripsi.csv")

## 2. Preprocessing Data

### 2.1. Normalisasi Teks

In [16]:
def clean_tweets(tweet):
    tweet = tweet.lower()

    # Menghapus mention, hastag, link 
    tweet = re.sub('(?:@\S*|#\S*|http(?=.*://)\S*)',' ',tweet)

    # Menghapus emoji
    tweet=re.sub("\\\\(?:x\w\w)","",tweet)
    
    # Mengubah non-ASCII dengan spasi
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    
    # Menghapus tanda baca
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    
    # Menghapus tags
    tweet=re.sub("&lt;/?.*?&gt;","&lt;&gt;",tweet)
    
    # Menghapus angka dan karakter spesial
    tweet=re.sub("(\\d|\\W)+"," ",tweet)

    # Menghapus simbol
    tweet = re.sub(r'â', '', tweet)
    tweet = re.sub(r'€', '', tweet)
    tweet = re.sub(r'¦', '', tweet)

    tweet = tweet.strip()

    return tweet

In [17]:
%%time
re_clean = df['text'].apply(lambda x: clean_tweets(x))
df['text'] = re_clean
df['text'] = df['text'].apply(lambda x: clean_tweets(x))

CPU times: user 29.4 ms, sys: 0 ns, total: 29.4 ms
Wall time: 30.5 ms


In [18]:
df.text.to_list()

['berdasarkan hasil peninjauan tokocrypto memutuskan untuk menghapus menghentikan perdagangan pada semua pasangan perdagangan untuk token berikut mulai november torn tornado cash xmr monero zec zcash salam to the moon',
 'tokocrypto platform perdagangan aset kripto itu mengumumkan mengurangi persen dari total karyawannya atau sekitar orang pada september',
 'wallet kepercayaan udah bisa busd bep tokocrypto belom nih',
 'tokocrypto hari ini umumkan akan hapus etf token link up link down trx up dan trx down tapi nanti pada hari senin oktober pukul wib etf token link up link down trx up dan trx down bakal dibuka sementara pada pukul wib agar bisa dijual',
 'hai kak kami informasikan bahwa tokocrypto mewajibkan bagi setiap nasabah untuk mengaktifkan fa ga dan sms per desember baik untuk penarikan fiat atau kripto untuk informasikan lengkap silakan klik halaman amel',
 'di binance link up dan down muncul kalau di tokocrypto ga muncul mungkin karena error seperti minggu lalu minggu lalu semu

### 2.2. Menghapus Karakter Berlebihan Pada Akhir Kata

In [19]:
def remove_same_character_last_words(textt):
    textt = textt.strip()
    textt = textt.lower()
    reverse_word = "".join(list(reversed(textt)))

    first_char = reverse_word[:1]

    n_iteration = 2

    same_char = 0

    while(first_char==reverse_word[(n_iteration-1):n_iteration]):
        same_char += 1
        n_iteration += 1

    new_text = reverse_word[same_char:]

    new_text = "".join(list(reversed(new_text)))

    return new_text

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
%%time
last_word_list = []
for text in df['text']:
    text = text.strip()
    temp_list = word_tokenize(text)
    temp_clean = [remove_same_character_last_words(word) for word in temp_list]
    last_word_list.append(" ".join(temp_clean))

df['text'] = last_word_list

CPU times: user 72.3 ms, sys: 5.09 ms, total: 77.4 ms
Wall time: 77.9 ms


In [22]:
df.text.to_list()

['berdasarkan hasil peninjauan tokocrypto memutuskan untuk menghapus menghentikan perdagangan pada semua pasangan perdagangan untuk token berikut mulai november torn tornado cash xmr monero zec zcash salam to the moon',
 'tokocrypto platform perdagangan aset kripto itu mengumumkan mengurangi persen dari total karyawannya atau sekitar orang pada september',
 'wallet kepercayaan udah bisa busd bep tokocrypto belom nih',
 'tokocrypto hari ini umumkan akan hapus etf token link up link down trx up dan trx down tapi nanti pada hari senin oktober pukul wib etf token link up link down trx up dan trx down bakal dibuka sementara pada pukul wib agar bisa dijual',
 'hai kak kami informasikan bahwa tokocrypto mewajibkan bagi setiap nasabah untuk mengaktifkan fa ga dan sms per desember baik untuk penarikan fiat atau kripto untuk informasikan lengkap silakan klik halaman amel',
 'di binance link up dan down muncul kalau di tokocrypto ga muncul mungkin karena error seperti minggu lalu minggu lalu semu

### 2.3. Stemming

In [23]:
%%time
factory = StemmerFactory()
stemmer = factory.create_stemmer()

list_stem = [stemmer.stem(str(txt)) for txt in df['text']]

CPU times: user 1min 42s, sys: 336 ms, total: 1min 43s
Wall time: 1min 49s


In [24]:
df['text'] = list_stem

In [25]:
df.text.to_list()

['dasar hasil tinjau tokocrypto putus untuk hapus henti dagang pada semua pasang dagang untuk token ikut mulai november torn tornado cash xmr monero zec zcash salam to the moon',
 'tokocrypto platform dagang aset kripto itu umum kurang persen dari total karyawannya atau sekitar orang pada september',
 'wallet percaya udah bisa busd bep tokocrypto bom nih',
 'tokocrypto hari ini umum akan hapus etf token link up link down trx up dan trx down tapi nanti pada hari senin oktober pukul wib etf token link up link down trx up dan trx down bakal buka sementara pada pukul wib agar bisa jual',
 'hai kak kami informasi bahwa tokocrypto wajib bagi tiap nasabah untuk aktif fa ga dan sms per desember baik untuk tari fiat atau kripto untuk informasi lengkap sila klik halaman amel',
 'di binance link up dan down muncul kalau di tokocrypto ga muncul mungkin karena error seperti minggu lalu minggu lalu semua token etf di tokocrypto sempat ga muncul tapi akhir pulih lagi saran saya kalau mau trading toke

2.4. Mengubah Slang Words

#### 2.4.1. Dua Dataset Slang di load dan dijadikan dictionary

Dataset 1

In [26]:
slang_word = pd.read_csv("colloquial-indonesian-lexicon.csv")
slang_word

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugime...,akronim,0,0
15002,gtau,enggak tau,0,Stidaknya mrka may berkarya Dan berusaha yg tr...,akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling...,reduplikasi,naturalisasi,0


In [27]:
dict_data_1 = dict(zip(slang_word.slang, slang_word.formal))
len(dict_data_1)

4331

Dataset 2

In [28]:
# Read Data
file_2  = open("combined_slang_words.txt", "r")
content2 = file_2.read()

# Transform into dictionary
slang_word_2 = ast.literal_eval(content2)
file_2.close()

dict_data_2 = slang_word_2

#### 2.3.2. Menggabungkan 2 Data Slang

In [29]:
print('Banyak baris pada dictionary 1: ', len(dict_data_1))
print('Banyak baris pada dictionary 2: ',len(dict_data_2))
print('-----------------------------------------')

dict3 = {**dict_data_1 , **dict_data_2}
print('Hasil Akhir panjang data slang: ',len(dict3))

Banyak baris pada dictionary 1:  4331
Banyak baris pada dictionary 2:  1018
-----------------------------------------
Hasil Akhir panjang data slang:  5042


In [30]:
dict3 = dict_data_2

#### 2.3.3. Mengubah Teks yang mengandung Slang pada Dictionary

In [31]:
word_changed = []
slang_changed = []
def slang_check(string):
    string = str(string)
    list_word =[]
    for word_text in string.split(' '):
        slang = dict3.get(word_text)
        if slang == None:
            list_word.append(word_text)
        else:
            list_word.append(slang)
            word_changed.append(word_text)
            slang_changed.append(slang)
    new_string = " ".join(list_word)
    return new_string

In [32]:
%%time
list_text_after_slang = list(map(lambda x: slang_check(x), df['text']))
df['text'] = list_text_after_slang
print('Jumlah Kata Terubah',len(word_changed))

Jumlah Kata Terubah 531
CPU times: user 5.73 ms, sys: 991 µs, total: 6.73 ms
Wall time: 7.1 ms


In [33]:
len(word_changed)

531

In [34]:
df.text.to_list()

['dasar hasil tinjau tokocrypto putus untuk hapus henti dagang pada semua pasang dagang untuk token ikut mulai november torn tornado cash xmr monero zec zcash salam to the moon',
 'tokocrypto platform dagang aset kripto itu umum kurang persen dari total karyawannya atau sekitar orang pada september',
 'wallet percaya sudah bisa busd bep tokocrypto bom ini',
 'tokocrypto hari ini umum akan hapus etf token link up link down trx up dan trx down tapi nanti pada hari senin oktober pukul wib etf token link up link down trx up dan trx down bakal buka sementara pada pukul wib agar bisa jual',
 'hai kak kami informasi bahwa tokocrypto wajib bagi tiap nasabah untuk aktif fa ga dan sms per desember baik untuk tari fiat atau kripto untuk informasi lengkap sila klik halaman amel',
 'di binance link up dan down muncul kalau di tokocrypto ga muncul mungkin karena error seperti minggu lalu minggu lalu semua token etf di tokocrypto sempat ga muncul tapi akhir pulih lagi saran saya kalau mau trading tok

### 2.4. Menghapus Stopwords

In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
stopw = stopwords.words('indonesian')
len(stopw)

758

In [37]:
# Menghapus kata negasi pada stopword
negasi = ["bukan", "bukanlah", "enggak", "kurang", "tidak","tidaklah", "anti"]
for word in stopw:
    if word in negasi:
        stopw.remove(word)
len(stopw)

752

In [38]:
def remove_stop(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopw]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence

In [39]:
df['text'] = [remove_stop(x) for x in df['text']]

In [40]:
df.text.to_list()

['dasar hasil tinjau tokocrypto putus hapus henti dagang pasang dagang token november torn tornado cash xmr monero zec zcash salam to the moon',
 'tokocrypto platform dagang aset kripto kurang persen total karyawannya orang september',
 'wallet percaya busd bep tokocrypto bom',
 'tokocrypto hapus etf token link up link down trx up trx down senin oktober wib etf token link up link down trx up trx down buka wib jual',
 'hai kak informasi tokocrypto wajib nasabah aktif fa ga sms desember tari fiat kripto informasi lengkap sila klik halaman amel',
 'binance link up down muncul tokocrypto ga muncul error minggu minggu token etf tokocrypto ga muncul pulih saran trading token up down gitu fokus btc eth up down',
 'phk massal fenomena tidak hindar sistem kapitalisme serba serbi mmc usaha shope tokocrypto indosat laku mangkas karyawannya tindak klaim upaya strategi efisiensi',
 'trust walletku isi iya habis win terima trust wallet tokocrypto',
 'wd tokocrypto ribu bukan rb',
 'pakai cryptowalle

### 2.5. Data Splitting

In [41]:
df_train, df_test = train_test_split(df, stratify=df['label_target'], test_size = 0.2, random_state=42)
print('Banyaknya baris train: ', len(df_train))
print('Banyaknya baris test: ', len(df_test))

print('------------------------------------')
print('Pembagian label pada data train')
print( df_train['label_target'].value_counts())

Banyaknya baris train:  436
Banyaknya baris test:  110
------------------------------------
Pembagian label pada data train
0    218
1    218
Name: label_target, dtype: int64


In [42]:
df_train, df_valid = train_test_split(df_train, stratify=df_train['label_target'], test_size = 0.1, random_state=42)
print('Banyaknya baris train: ', len(df_train))
print('Banyaknya baris valid: ', len(df_valid))

print('------------------------------------')
print('Pembagian label pada data train')
print( df_train['label_target'].value_counts())

Banyaknya baris train:  392
Banyaknya baris valid:  44
------------------------------------
Pembagian label pada data train
1    196
0    196
Name: label_target, dtype: int64


In [43]:
#Save Dataset

df_train.to_csv(r'C:\Users\DaMxNeD\Downloads\train_non_slang.csv', index=False)
df_test.to_csv(r'C:\Users\DaMxNeD\Downloads\test_non_slang.csv', index=False)
df_valid.to_csv(r'C:\Users\DaMxNeD\Downloads\valid_non_slang.csv', index=False)

In [44]:
#Save Dataset

df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)
df_valid.to_csv('valid.csv', index=False)

In [45]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
#df_train['text'].to_list()

In [46]:
df_train

,text,label_target
0,pakai indodax tokocrypto fe nya geda ga,1
1,aplikasi friendly tampil ui simple mudah user ...,1
2,mengunakan trading dodo fe murah fairing yuk d...,1
3,min koin nagus zcash limit koin jt saing bitcoin,0
4,pakai apk pintu tokocrypto tinggal klik jual s...,0
...,...,...
387,yuk beli bnb market token lunc bttc usdt shib,1
388,new earnings fre busd airdrop in tokocrypto wa...,1
389,pegang token min,1
390,buka ga nunggu tanggal etf link parah woy,0


## 3. Data Modelling

### 3.1 Import data ke pytorch

In [47]:
folder_data = 'Data'
folder_model = 'Model'

In [48]:
!python -m spacy download xx_sent_ud_sm

2022-11-16 03:41:49.134909: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 28.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('xx_sent_ud_sm')


In [49]:
folder_data = 'Data'
folder_model = 'Model'

# Labelling pada Pytorch
field_label = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
field_text = Field(tokenize='spacy',tokenizer_language='xx_sent_ud_sm', lower=True, include_lengths=True, batch_first=True)
fields = [ ('text', field_text), ('label_target', field_label)]

# Mengambill data
#tab_train, tab_valid, tab_test = TabularDataset.splits(path=folder_data, train='train_non_slang.csv', validation='valid_non_slang.csv', test='test_non_slang.csv',
#                                           format='CSV', fields=fields, skip_header=True)
tab_train, tab_valid, tab_test = TabularDataset.splits(path = './', train='train.csv', validation='valid.csv', test='test.csv', format='CSV', fields=fields, skip_header=True)

train_data = BucketIterator(tab_train, batch_size=32, sort_key=lambda a: len(a.text),
                            device=device, sort=True, sort_within_batch=True)
valid_data = BucketIterator(tab_valid, batch_size=32, sort_key=lambda a: len(a.text),
                            device=device, sort=True, sort_within_batch=True)
test_data = BucketIterator(tab_test, batch_size=32, sort_key=lambda a: len(a.text),
                            device=device, sort=True, sort_within_batch=True)

#Membuat Vocab
field_text.build_vocab(tab_train, min_freq=3)

In [50]:
train_data.len

AttributeError: ignored

In [ ]:
print('Banyaknya row data train: ', len(tab_train))

In [ ]:
print(vars(tab_train.examples[0]))

In [ ]:
print(vars(tab_test.examples[44]))

In [ ]:
print(vars(tab_test.examples[44]))

In [ ]:
label_size = {'Negative': 0, 'Positive':0}
for i in range(len(tab_train)):
    label_temp = vars(tab_train[i])['label_target']
    if label_temp == '1':
        label_size['Positive'] += 1
    else:
        label_size['Negative'] += 1
    
print(label_size)

In [ ]:
max_len_text = 0
for i in range(len(tab_train)):
    text_temp = vars(tab_train[i])['text']
    if len(text_temp) > max_len_text:
        max_len_text = len(text_temp)

print('Maksimal panjang teks', max_len_text)

In [ ]:
print(field_text.vocab.freqs.most_common(20))

In [ ]:
len(field_text.vocab)

In [ ]:
field_text.vocab.stoi['trading']

In [ ]:
field_text.vocab.itos[:10]

In [ ]:
field_text.vocab.itos[18]

In [ ]:
print(vars(tab_train.examples[0]))

In [ ]:
for ((text, len_text), label), _ in train_data:
    print('ROW----------------------------------------------------')
    print(text.shape)
    print("label", label)
    print("text", text, 'Lenn', len_text)
    print("___", _)

### 3.2 Membuat Model LSTM

In [ ]:
torch.__version__

In [ ]:
class LSTM(nn.Module):
    def __init__(self, hidden_layer=128, lstm_layer= 1, embedding_layer = 300):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(len(field_text.vocab), embedding_layer)
        self.dimension = hidden_layer
        self.lstm = nn.LSTM(input_size=embedding_layer,
                            hidden_size=hidden_layer,
                            num_layers=lstm_layer,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*hidden_layer, 1)

    def forward(self, text, len_text):

        #Word Embedding
        text_embedding = self.embedding(text)

        #Masuk ke layer LSTM
        input_packed = pack_padded_sequence(text_embedding, len_text.to('cpu'), batch_first=True, enforce_sorted=False)
        ouput_packed, _ = self.lstm(input_packed)
        output_lstm_all, _ = pad_packed_sequence(ouput_packed, batch_first=True)

        output_lstm_forward = output_lstm_all[range(len(output_lstm_all)), len_text - 1, : self.dimension]
        output_lstm_reverse = output_lstm_all[:, 0, self.dimension:]
        output_concat = torch.cat((output_lstm_forward, output_lstm_reverse), 1)
        
        output_dropped = self.drop(output_concat)

        #With drop
        text_fc = self.fc(output_dropped)
        text_squeeze = torch.squeeze(text_fc, 1)
        final_output = torch.sigmoid(text_squeeze) 

        return final_output

In [ ]:
def save_history_loss(folder_save, list_train_loss, list_valid_loss, global_steps_list):

    if folder_save == None:
        return
    
    history_loss = {'list_train_loss': list_train_loss,
                  'list_valid_loss': list_valid_loss,
                  'global_steps_list': global_steps_list}
    
    torch.save(history_loss, folder_save)


def load_history_loss(folder_load):

    if folder_load == None:
        return
    
    history_loss = torch.load(folder_load, map_location=device)
    
    return history_loss['list_train_loss'], history_loss['list_valid_loss'], history_loss['global_steps_list']

### 3.3 Membuat Fungsi Training

In [ ]:
def model_run_train(model_train,
          optimizer_train,
          criterion = nn.BCELoss(),
          train_dataloader = train_data,
          valid_dataloader = valid_data,
          num_epochs = 16,
          mode_gridsearch = False,
          eval_half = len(train_data) // 2,
          folder_path = folder_model ):
    
    # Inisialisasi
    list_train_loss = []
    list_valid_loss = []
    global_steps_list = []

    train_loss_running = 0.0
    valid_loss_running = 0.0
    global_steps = 0
    total_global_steps = num_epochs*len(train_dataloader)


    # Pelatihan pada looping
    model_train.train()
    for epoch in range(num_epochs):
        for ((text, len_text), labels), _ in train_dataloader:
       
            text = text.to(device)
            len_text = len_text.to(device)
            labels = labels.to(device)   
            
            # Pelatihan model dengan data tersedia
            output_model = model_train(text, len_text)
            loss = criterion(output_model, labels)

            optimizer_train.zero_grad()
            loss.backward()
            optimizer_train.step()

            # Update nilai Loss
            train_loss_running += loss.item()
            global_steps += 1


            # Tahap evaluasi terhadap valid data
            if global_steps % eval_half == 0:
                model_train.eval()
                with torch.no_grad():                    
                  # Valid data
                  for ((text, len_text), labels ), _ in valid_dataloader:
                      labels = labels.to(device)
                      text = text.to(device)
                      len_text = len_text.to(device)
                      
                      output_model = model_train(text, len_text)
                      loss = criterion(output_model, labels)

                      valid_loss_running += loss.item()

                # Evaluasi
                avg_train_loss = train_loss_running / eval_half
                avg_valid_loss = valid_loss_running / len(valid_dataloader)
                
                list_train_loss.append(avg_train_loss)
                list_valid_loss.append(avg_valid_loss)
                global_steps_list.append(global_steps)

                # Mengubah ke nilai awal
                train_loss_running = 0.0                
                valid_loss_running = 0.0
                model_train.train()

                # Print untuk Iterasi
                if mode_gridsearch == True:
                    continue
                else:
                    print(f'Epoch [{epoch+1}/{num_epochs}] | Global Steps [{global_steps}/{total_global_steps}] | Train Loss: {avg_train_loss:.4f} | Valid Loss: {avg_valid_loss:.4f}')
                
    save_history_loss('metrics.pt', list_train_loss, list_valid_loss, global_steps_list)

    return list_valid_loss[-1], list_train_loss[-1]

In [ ]:
%%time
torch.manual_seed(42)
model = LSTM(hidden_layer=256, lstm_layer=3, embedding_layer=100).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
epoch = 16

model_run_train(model_train=model, optimizer_train=optimizer, num_epochs=epoch)

### 3.4 Visualisasi pada model loss

In [ ]:
list_train_loss, list_valid_loss, global_steps_list = load_history_loss('metrics.pt')
plt.plot(global_steps_list, list_train_loss, label='Train')
plt.plot(global_steps_list, list_valid_loss, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 

In [ ]:
temp_train = []
temp_valid = []
for i in range(len(list_train_loss)):
    if (i+1) % 2 != 0:
        continue
    temp_train.append(list_train_loss[i])
    temp_valid.append(list_valid_loss[i])

ep_list = range(1, epoch+1)

plt.plot(ep_list, temp_train, label='Train')
plt.plot(ep_list, temp_valid, label='Valid')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show() 

### 3.5 Evaluasi Data Testing

In [ ]:
# Evaluation Function

def model_evaluate(model, test_loader, mode_gridsearch = False):
    threshold=0.5
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for ((text, len_text), labels), _ in test_loader:           
            labels = labels.to(device)
            text = text.to(device)
            len_text = len_text.to(device)
            
            output_model = model(text, len_text)

            # 0.5 keatas menjadikan label 1
            output_model = (output_model > threshold).int()
            
            y_pred.extend(output_model.tolist())
            y_true.extend(labels.tolist())

    if mode_gridsearch == True :
        return f1_score(y_true, y_pred), precision_score(y_true, y_pred, zero_division=0), recall_score(y_true, y_pred)
    else:
        print('Classification Report:')
        #print(accuracy_score(y_true, y_pred))
        print(f1_score(y_true, y_pred))
        #print(precision_score(y_true,y_pred))
        #print(recall_score(y_true,y_pred))
        print(classification_report(y_true, y_pred, labels=[1,0], digits=4))
        
        ax= plt.subplot()
        cm = confusion_matrix(y_true, y_pred, labels=[1,0])
        sns.heatmap(cm, annot=True, ax = ax, fmt="d")

        ax.xaxis.set_ticklabels(['Positive', 'Negative'])
        ax.yaxis.set_ticklabels(['Positive', 'Negative'])

        ax.set_title('Confusion Matrix')

        ax.set_xlabel('Predicted Labels')
        ax.set_ylabel('True Labels')

        #for i in range(len(y_pred)):
        #    print(f'{y_pred[i]} | {y_true[i]}')
        #print(y_pred)
        #print('------------------------')
        #print(y_true)
   
        return

In [ ]:
model_evaluate(model, test_data)

### 3.6 Gridsearch Hyperparameter Tuning

In [ ]:
def GridSearchTrain(hidden_dim_grid, lstm_layer_grid, lr_grid, epoch_grid, embed_dim_grid):
    #-------------------------------------------------------------------------
    #Adam
    torch.manual_seed(42)
    model_adam = LSTM(hidden_layer=hidden_dim_grid, lstm_layer=lstm_layer_grid, embedding_layer=embed_dim_grid).to(device)
    optimizer_adam = optim.Adam(model_adam.parameters(), lr=lr_grid)

    valid_loss_adam, train_loss_adam = model_run_train(model_train=model_adam, optimizer_train=optimizer_adam, num_epochs=epoch_grid, mode_gridsearch=True)

    f1_score_adam, precision_adam, recall_adam = model_evaluate(model_adam, test_data, mode_gridsearch=True)


    #-------------------------------------------------------------------------
    #SGD
    torch.manual_seed(42)
    model_sgd = LSTM(hidden_layer=hidden_dim_grid, lstm_layer=lstm_layer_grid, embedding_layer=embed_dim_grid).to(device)
    optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=lr_grid)

    valid_loss_sgd, train_loss_sgd = model_run_train(model_train=model_sgd, optimizer_train=optimizer_sgd, num_epochs=epoch_grid, mode_gridsearch=True)

    f1_score_sgd, precision_sgd, recall_sgd = model_evaluate(model_sgd, test_data, mode_gridsearch=True)

    #-------------------------------------------------------------------------
    best_param = [hidden_dim_grid, lstm_layer_grid, lr_grid, epoch_grid, embed_dim_grid]

    if f1_score_adam > f1_score_sgd:
        best_param.append('Adam')
        return f1_score_adam, precision_adam, recall_adam, valid_loss_adam, train_loss_adam, best_param
    elif f1_score_adam < f1_score_sgd:
        best_param.append('SGD')
        return f1_score_sgd, precision_sgd, recall_sgd, valid_loss_sgd, train_loss_sgd, best_param
    elif f1_score_adam == f1_score_sgd:
        if valid_loss_adam < valid_loss_sgd:
            best_param.append('Adam')
            return f1_score_adam, precision_adam, recall_adam, valid_loss_adam, train_loss_adam, best_param
        else:
            best_param.append('SGD')
            return f1_score_sgd, precision_sgd, recall_sgd, valid_loss_sgd, train_loss_sgd, best_param


In [ ]:
%%time
embedding_dim_param_list = [50, 100, 200, 300, 400]
hidden_dim_param_list = [16, 32, 64, 128, 256]
lstm_layer_param_list = [1, 2, 3]
lr_param_list = [0.0001, 0.0005, 0.001, 0.005, 0.01]
epoch_param_list = [2, 4, 8, 16, 32]


# variabel f1 score terbaik
temp_best_score = 0
temp_best_val_loss = 999
temp_best_param = []


# iteration counter
iter_num = 0
max_iter = len(embedding_dim_param_list) * len(hidden_dim_param_list) * len(lstm_layer_param_list) * len(lr_param_list) * len(epoch_param_list)

# membuat dokumentasi csv
csvFile = open('Result_Gridsearch_model.csv', 'a', encoding = 'utf-8') #creates a file in which you want to store the data.
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['Hidden Layer', 'LSTM Layer', 'Learning Rate', 'Epoch', 'Embedding Layer', 'Optimizer', 'Train Loss', 'Val Loss', 'Precision', 'Recall', 'F1 Score']) 


for embedding_dim_temp in embedding_dim_param_list:
    for hidden_dim_temp in hidden_dim_param_list:
        for lstm_layer_temp in lstm_layer_param_list:
            for lr_temp in lr_param_list:
                for epoch_temp in epoch_param_list:
                    score, precision, recall, val_loss, train_loss, params = GridSearchTrain(hidden_dim_temp, lstm_layer_temp, lr_temp, epoch_temp, embedding_dim_temp)
                    
                    # Menambahkan hasil ke csv
                    csvWriter.writerow([params[0], params[1], params[2], params[3], params[4], params[5], train_loss, val_loss, precision, recall, score]) 

                    iter_num += 1
                    print(f'iteration done: {iter_num}/{max_iter}')

                    if score > temp_best_score:
                        temp_best_score = score
                        temp_best_val_loss = val_loss
                        temp_best_param = params
                    elif score == temp_best_score:
                        if val_loss < temp_best_val_loss:
                            temp_best_score = score
                            temp_best_val_loss = val_loss
                            temp_best_param = params

print('----------------------------------')
print('Best Model: ')
print('f1-score: ', temp_best_score)
print('val loss: ', temp_best_val_loss)
print('params: ', temp_best_param)

csvFile.close()